In [8]:
import numpy as np
import pandas as pd

In [9]:
ratings = pd.read_csv('../data/reduced/ratings_m10.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
ratings.reindex()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
81111,610,159093,3.0,1493847704
81112,610,164179,5.0,1493845631
81113,610,166528,4.0,1493879365
81114,610,168250,5.0,1494273047


In [11]:
movies = pd.read_csv('../data/reduced/movies_m10_rich_pre.csv', index_col='movieId')
movies.head()

,tmdbId,imdbId,cast,director,keywords,overview,title,genres,year
movieId,,,,,,,,,
117529,135397,369610,"['Chris Pratt', 'Bryce Dallas Howard', 'Irrfan...",Colin Trevorrow,"['monster', 'dna', 'tyrannosaurus rex', 'veloc...",Twenty-two years after the events of Jurassic ...,Jurassic World,"['Action', 'Adventure', 'Drama', 'Sci-Fi', 'Th...",2015
122882,76341,1392190,"['Tom Hardy', 'Charlize Theron', 'Hugh Keays-B...",George Miller,"['future', 'chase', 'post-apocalyptic', 'dysto...",An apocalyptic story set in the furthest reach...,Mad Max: Fury Road,"['Action', 'Adventure', 'Sci-Fi', 'Thriller']",2015
122886,140607,2488496,"['Harrison Ford', 'Mark Hamill', 'Carrie Fishe...",J.J. Abrams,"['android', 'spaceship', 'jedi', 'space opera'...",Thirty years after defeating the Galactic Empi...,Star Wars: Episode VII - The Force Awakens,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi', '...",2015
139385,281957,1663202,"['Leonardo DiCaprio', 'Tom Hardy', 'Will Poult...",Alejandro GonzÃ¡lez IÃ±Ã¡rritu,"['father-son relationship', 'rape', 'based on ...","In the 1820s, a frontiersman, Hugh Glass, sets...",The Revenant,"['Adventure', 'Drama']",2015
134130,286217,3659388,"['Matt Damon', 'Jessica Chastain', 'Kristen Wi...",Ridley Scott,"['based on novel', 'mars', 'nasa', 'isolation'...","During a manned mission to Mars, Astronaut Mar...",The Martian,"['Adventure', 'Drama', 'Sci-Fi']",2015


In [12]:
movies_title = movies[['title']]
movies_title.head()

,title
movieId,
117529,Jurassic World
122882,Mad Max: Fury Road
122886,Star Wars: Episode VII - The Force Awakens
139385,The Revenant
134130,The Martian


In [13]:
user_movie_rating = ratings.merge(movies_title, on='movieId')
user_movie_rating.head()

,userId,movieId,rating,timestamp,title
0,1,1,4.0,964982703,Toy Story
1,5,1,4.0,847434962,Toy Story
2,7,1,4.5,1106635946,Toy Story
3,15,1,2.5,1510577970,Toy Story
4,17,1,4.5,1305696483,Toy Story


In [14]:
user_movie_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75043 entries, 0 to 75042
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     75043 non-null  int64  
 1   movieId    75043 non-null  int64  
 2   rating     75043 non-null  float64
 3   timestamp  75043 non-null  int64  
 4   title      75043 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 2.9+ MB


In [15]:
user_movie_mat = user_movie_rating.pivot(index='movieId', columns='userId', values='rating').fillna(0)

In [16]:
user_movie_mat

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
139644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
140110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def corr_between_users(ratings: pd.DataFrame, user1: str, user2: str):
    rated_movies_by_both = ratings[[user1, user2]].dropna(axis=0).values
    user1_ratings = rated_movies_by_both[:, 0]
    user2_ratings = rated_movies_by_both[:, 1]
    return np.corrcoef(user1_ratings, user2_ratings)[0,1]

In [18]:
users_list = list(user_movie_mat.columns)
movies_list = list(user_movie_mat.index)

users_similarity_mat = np.array([[corr_between_users(user_movie_mat, user1, user2) for user1 in users_list] for user2 in users_list])
users_similarity_mat = pd.DataFrame(users_similarity_mat, index=users_list, columns=users_list)

In [19]:
users_similarity_mat

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,1.000000,-0.002590,0.101652,0.170002,0.097906,0.069130,0.121411,0.111981,0.035627,-0.045759,...,0.048202,0.125364,0.222054,0.025159,0.102362,0.110384,0.229172,0.202948,0.090650,0.041636
2,-0.002590,1.000000,-0.007923,-0.024640,0.003562,-0.004697,0.004426,0.012476,-0.012090,0.061093,...,0.232685,-0.008019,-0.038886,-0.022777,-0.030848,-0.014936,-0.016342,-0.013217,0.021572,0.098181
3,0.101652,-0.007923,1.000000,-0.013336,0.000403,-0.014649,-0.017258,-0.001189,-0.007609,-0.015254,...,-0.002512,-0.007307,0.037271,-0.014334,0.004831,-0.002996,0.017235,0.000902,-0.008132,0.020283
4,0.170002,-0.024640,-0.013336,1.000000,0.104839,0.033633,0.078901,0.041316,-0.010483,-0.023073,...,0.016703,0.102126,0.294236,0.006432,0.052021,0.116328,0.084298,0.047905,0.017784,0.003049
5,0.097906,0.003562,0.000403,0.104839,1.000000,0.314069,0.093754,0.446437,-0.014746,0.009821,...,0.060363,0.448747,0.101631,0.264069,0.127826,0.075770,0.145223,0.086493,0.316657,0.010844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.110384,-0.014936,-0.002996,0.116328,0.075770,-0.010581,0.199735,0.082817,0.078327,0.042217,...,0.153766,0.057460,0.264562,-0.012448,0.094888,1.000000,0.108194,0.129108,0.064564,0.051476
607,0.229172,-0.016342,0.017235,0.084298,0.145223,0.118561,0.154812,0.167234,-0.013855,-0.046974,...,0.068037,0.164199,0.189427,0.107868,0.078744,0.108194,1.000000,0.187341,0.164677,0.043129
608,0.202948,-0.013217,0.000902,0.047905,0.086493,0.044126,0.283628,0.143291,0.068830,-0.027472,...,0.099449,0.099922,0.108195,0.078609,0.084657,0.129108,0.187341,1.000000,0.106755,0.151827
609,0.090650,0.021572,-0.008132,0.017784,0.316657,0.272152,0.089062,0.489783,-0.012410,0.005897,...,0.027714,0.396067,0.030669,0.286018,0.108746,0.064564,0.164677,0.106755,1.000000,0.021997


In [27]:
def get_rated_user_for_a_movie(ratings_df: pd.DataFrame, movie: int):
    return ratings_df.loc[movie, :].dropna().index.values


def get_top_neighbors(
    similarity_df: pd.DataFrame, user: str, rated_users: str, n_neighbors: int
):
    return similarity_df[user][rated_users].nlargest(n_neighbors).to_dict()


def subtract_bias(rating: float, mean_rating: float):
    return rating - mean_rating


def get_neighbor_rating_without_bias_per_movie(
    ratings_df: pd.DataFrame, user: str, movie: int
):
    """Substract the rating of a user from the mean rating of that user to eliminate bias"""
    mean_rating = ratings_df[user].mean()
    rating = ratings_df.loc[movie, user]
    return subtract_bias(rating, mean_rating)


def get_ratings_of_neighbors(ratings_df: pd.DataFrame, neighbors: list, movie: int):
    """Get the ratings of all neighbors after adjusting for biases"""
    return [
        get_neighbor_rating_without_bias_per_movie(ratings_df, neighbor, movie)
        for neighbor in neighbors
    ]

def get_weighted_average_rating_of_neighbors(ratings: list, neighbor_distance: list):
    weighted_sum = np.array(ratings).dot(np.array(neighbor_distance))
    abs_neigbor_distance = np.abs(neighbor_distance)
    return weighted_sum / np.sum(abs_neigbor_distance)


def ger_user_rating(ratings_df: pd.DataFrame, user: str, avg_neighbor_rating: float):
    user_avg_rating = ratings_df[user].mean()
    return round(user_avg_rating + avg_neighbor_rating, 2)

In [32]:
def predict_rating(
    df: pd.DataFrame,
    similarity_df: pd.DataFrame,
    user: int,
    movie: int,
    n_neighbors: int = 2,
):
    """Predict the rating of a user for a movie based on the ratings of neighbors"""
    ratings_df = df.copy()

    rated_users = get_rated_user_for_a_movie(ratings_df, movie)

    top_neighbors_distance = get_top_neighbors(
        similarity_df, user, rated_users, n_neighbors
    )
    neighbors, distance = top_neighbors_distance.keys(), top_neighbors_distance.values()

    print(f"Top {n_neighbors} neighbors of user {user}, {movie}: {list(neighbors)}")

    ratings = get_ratings_of_neighbors(ratings_df, neighbors, movie)
    avg_neighbor_rating = get_weighted_average_rating_of_neighbors(
        ratings, list(distance)
    )

    return ger_user_rating(ratings_df, user, avg_neighbor_rating)


In [33]:
movie = 1 #toy story
user1 = 1
rating = predict_rating(user_movie_mat, users_similarity_mat, user1, movie)
rating

Top 2 neighbors of user 1, 1: [1, 266]


3.52